In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
#Parametros de ejecucion del programa
nombreArchivoDataset = "./Datos en Pasos Intermedios/Dataset Reconstruido.csv"
nombreColumnaDuplicadaSalida = "Interested in buying a mobile home policy"
nombreColumnaSalida = "Number of mobile home policies"
nombreColumnaFase = "Phase"
etiquetaDatosEntrenamiento = "Training"
etiquetaDatosPrueba = "Test"

colorTextoVerde = (35, 155, 86)
colorTextoRojo = (176, 58, 46)

In [3]:
def printColouredText(text, foregroundColor = (0, 0, 0), backgroundColor = (255, 255, 255)):
    return "\033[48;2;{};{};{}m\033[38;2;{};{};{}m{}\033[38;2;0;0;0m\033[48;2;255;255;255m".format(backgroundColor[0], backgroundColor[1], backgroundColor[2], foregroundColor[0], foregroundColor[1], foregroundColor[2], text)

In [4]:
fullDatasetFile = pd.read_csv(nombreArchivoDataset, sep = ";", decimal = ".")
fullDatasetFile[nombreColumnaDuplicadaSalida] = fullDatasetFile[nombreColumnaSalida].tolist()
fullDatasetFile.replace({nombreColumnaDuplicadaSalida: {0: "No", 1: "Yes"}}, inplace = True)

#Filtrado de informacion para separar el dataset de entrenamiento y el dataset de prueba del dataset completo
trainingDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosEntrenamiento)]
testDatasetFile = fullDatasetFile[fullDatasetFile[nombreColumnaFase].str.contains(etiquetaDatosPrueba)]

print("Se ha extraído la información del archivo", printColouredText(nombreArchivoDataset, colorTextoRojo))
print("Se tiene(n)", printColouredText(str(len(fullDatasetFile.index)) + " dato(s)", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(trainingDatasetFile.index)) + " dato(s) de entrenamiento", colorTextoVerde))
print("Se tiene(n) de", printColouredText(str(len(testDatasetFile.index)) + " dato(s) de prueba", colorTextoVerde))

Se ha extraído la información del archivo ./Datos en Pasos Intermedios/Dataset Reconstruido.csv
Se tiene(n) 9822 dato(s)
Se tiene(n) de 5822 dato(s) de entrenamiento
Se tiene(n) de 4000 dato(s) de prueba


In [5]:
fullDatasetFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9822 entries, 0 to 9821
Data columns (total 88 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Customer Subtype                                  9822 non-null   int64 
 1   Number of houses                                  9822 non-null   int64 
 2   Avg size household                                9822 non-null   int64 
 3   Avg age                                           9822 non-null   int64 
 4   Customer main type                                9822 non-null   int64 
 5   Roman catholic                                    9822 non-null   int64 
 6   Protestant                                        9822 non-null   int64 
 7   Other religion                                    9822 non-null   int64 
 8   No religion                                       9822 non-null   int64 
 9   Married                       

In [6]:
for col in fullDatasetFile.columns:
    if col != nombreColumnaSalida and col != nombreColumnaFase:
        print(col)
        break

pivotTableWith2DetailLevels = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = col, columns = [nombreColumnaFase, nombreColumnaDuplicadaSalida], aggfunc = 'count')
pivotTableWith2DetailLevels.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel = pd.pivot_table(fullDatasetFile, values = nombreColumnaSalida, index = col, columns = nombreColumnaDuplicadaSalida, aggfunc = 'count')
pivotTableWith1DetailLevel.fillna(0.0, inplace = True)
pivotTableWith1DetailLevel["General Total"] = pivotTableWith1DetailLevel[pivotTableWith1DetailLevel.columns].sum(axis = 1)
pivotTableWith1DetailLevel.rename(columns = {tableColumn: ("", tableColumn) for tableColumn in pivotTableWith1DetailLevel.columns}, inplace = True)
fullPivotTable = pd.concat([pivotTableWith2DetailLevels, pivotTableWith1DetailLevel], axis = 1)

suma = fullPivotTable.sum()
suma.name = "General Total"
pivotTableSummarizedByRows = pd.DataFrame(suma.transpose(), columns = fullPivotTable.columns, index = [suma.name])
pivotTableSummarizedByRows.loc[suma.name] = suma.tolist()
fullPivotTable = pd.concat([fullPivotTable, pivotTableSummarizedByRows])

for column in fullPivotTable.columns:
    fullPivotTable[column] = [int(value) for value in fullPivotTable[column].tolist()]

#x1.info()
display(fullPivotTable)

Customer Subtype


Phase                                     Phase 1 [Training]       \
Interested in buying a mobile home policy                 No  Yes   
1                                                        111   13   
2                                                         76    6   
3                                                        224   25   
4                                                         50    2   
5                                                         43    2   
6                                                        107   12   
7                                                         41    3   
8                                                        288   51   
9                                                        266   12   
10                                                       156    9   
11                                                       144    9   
12                                                        95   16   
13                                                       166   13   
15                                                         5    0   
16                                                        16    0   
17                                                         9    0   
18                                                        19    0   
19                                                         3    0   
20                                                        23    2   
21                                                        15    0   
22                                                        94    4   
23                                                       247    4   
24                                                       175    5   
25                                                        80    2   
26                                                        47    1   
27                                                        49    1   
28                                                        25    0   
29                                                        84    2   
30                                                       114    4   
31                                                       199    6   
32                                                       133    8   
33                                                       764   46   
34                                                       173    9   
35                                                       206    8   
36                                                       209   16   
37                                                       122   10   
38                                                       316   23   
39                                                       309   19   
40                                                        71    0   
41                                                       200    5   
General Total                                           5474  348   

Phase                                     Phase 2 [Test]                  \
Interested in buying a mobile home policy             No  Yes    No  Yes   
1                                                     81   13   192   26   
2                                                     61    5   137   11   
3                                                    176    8   400   33   
4                                                     37    1    87    3   
5                                                     25    0    68    2   
6                                                     76   14   183   26   
7                                                     26    2    67    5   
8                                                    186   21   474   72   
9                                                    177    5   443   17   
10                                                    92   14   248   23   
11                                                   126    7   270   16   
12                                          